In [1]:
from langgraph.graph import StateGraph,START,END
from langgraph.graph.message import add_messages
from typing import Annotated, TypedDict
from langchain_openai import ChatOpenAI
from langchain_teddynote.tools.tavily import TavilySearch

from langchain_teddynote.graphs import visualize_graph
from langgraph.prebuilt import ToolNode, tools_condition



## 도구 사용 분기 처리

1. state 생성
2. llm 생성
3. 도구 리스트 생성
4. llm과 도구 연결
5. 노드 생성
6. 노드 추가
7. 노드간 엣지 추가
8. 분기 조건처리
9. 그래프 확인

In [10]:

# 1. state
class State(TypedDict):
    messages : Annotated[list,add_messages]

# 2. llm 
llm = ChatOpenAI(model="gpt-4.1-mini",temperature=0)

# 3. 도구 생성
tool = TavilySearch(max_results=3)
tools = [tool]
tool_node = ToolNode( [tool])

tools(tags=None, recurse=True, explode_args=False, func_accepts={'config': ('N/A', <class 'inspect._empty'>), 'store': ('store', None)}, tools_by_name={'tavily_web_search': TavilySearch(client=<tavily.tavily.TavilyClient object at 0x72a0fbb63ed0>)}, tool_to_state_args={'tavily_web_search': {}}, tool_to_store_arg={'tavily_web_search': None}, handle_tool_errors=True, messages_key='messages')

In [9]:

# 4. llm 과 구 연결
llm_bind_tool = llm.bind_tools(tools)
context = llm_bind_tool.invoke('삼성전자')
print(context)

content='' additional_kwargs={'tool_calls': [{'id': 'call_Ap81W3GSXwE199S3xDRXUw4d', 'function': {'arguments': '{"query":"삼성전자"}', 'name': 'tavily_web_search'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 94, 'total_tokens': 113, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6d7dcc9a98', 'id': 'chatcmpl-CJYlSbe5pUEGCznCgH9YK8w9mwnVP', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--1418d2bb-951e-46d3-9972-c71b0c6b8a31-0' tool_calls=[{'name': 'tavily_web_search', 'args': {'query': '삼성전자'}, 'id': 'call_Ap81W3GSXwE199S3xDRXUw4d', 'type': 'tool_call'}] usage_metadata={'input_tokens': 94, 'output_tokens': 19, 'total_tokens': 113, 'input_token_details': {'audio'

In [ ]:

# 5. 노드 생성
def chatbot(state:State):
    # print(state[('messages')])
    return State({'messages':[llm_bind_tool.invoke(state['messages'])]})

# 8. 분기 처리
def route_tool_use(state:State):
    if messages := state.get("messages", []):
        # 가장 최근 AI 메시지 추출
        ai_message = messages[-1]
    else:
        # 입력 상태에 메시지가 없는 경우 예외 발생
        raise ValueError(f"No messages found in input state to tool_edge: {state}")

    # AI 메시지에 도구 호출이 있는 경우 "tools" 반환
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        # 도구 호출이 있는 경우 "tools" 반환
        return "tool_node"
    # 도구 호출이 없는 경우 "END" 반환
    return END
    

# 6. 노드 추가
state_graph = StateGraph(State)
state_graph.add_node('chatbot',chatbot)
state_graph.add_node('tool_node',tool_node)
# 7. 엣지 추가
state_graph.add_edge(START,'chatbot')
# 8. 분기 처리
state_graph.add_conditional_edges(
    source='chatbot',
    path=route_tool_use,
    path_map={
        'tool_node':'tool_node',
        END:END
    }
)
# 위에랑 같은 처리 route_tool_use 직접 구현없이 tools_condition으로 구현가능
# state_graph.add_conditional_edges(
#     source='chatbot',
#     path=tools_condition
# )
state_graph.add_edge('tool_node','chatbot')
state_graph.add_edge('chatbot',END)


In [4]:

graph = state_graph.compile()


# 그래프 시각화
visualize_graph(graph)

그래프 시각화 실패 (추가 종속성 필요): Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`
ASCII로 그래프 표시:
          +-----------+             
          | __start__ |             
          +-----------+             
                 *                  
                 *                  
                 *                  
            +---------+             
            | chatbot |             
            +---------+             
          ...         ...           
         .               .          
       ..                 ..        
+---------+           +-----------+ 
| __end__ |           | tool_node | 
+---------+           +---------

In [5]:
graph.invoke({"messages":"오늘의뉴스"})

{'messages': [HumanMessage(content='오늘의뉴스', additional_kwargs={}, response_metadata={}, id='24ceda53-6b21-4d71-b525-22713de9ee3b'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_85ofD8I1lUYMJ7KwIoV2YNo7', 'function': {'arguments': '{"query":"오늘의 뉴스"}', 'name': 'tavily_web_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 94, 'total_tokens': 113, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6d7dcc9a98', 'id': 'chatcmpl-CJYWZuKr8jRl6Dh4qkVSZg8LEef0b', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--047faf7a-6f21-417a-8e39-dff362b831fc-0', tool_calls=[{'name': 'tavily_web_search', 'args': {'query': '오늘의 뉴스'}, 'id': 'call_85ofD8